# EES1133 Lab 2b: 30-Layer Grey Atmosphere Model with `climlab`

Adapted from notes by:
[Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany

In [1]:
#  Load packages that we will need

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import climlab

In this lab and future labs, we will work with `climlab` in order to test different climate modelling concepts using simple radiative equilibrium (RE) and radiative-convective equilibrium (RCE) models. 

____________
<a id='section2'></a>
## I. The observed annual, global mean temperature profile
____________

To start, we want to model the OLR in a column whose temperatures match observations. First, we'll calculate the global, annual mean air temperature from the NCEP Reanalysis data.

In [3]:
# This will try to read the data over the internet.
ncep_filename = 'air.mon.1981-2010.ltm.nc'
#  to read over internet
ncep_url = "http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/"

#  Open handle to data
ncep_air = xr.open_dataset( ncep_url + ncep_filename, decode_times=False )

In [4]:
#  Take global, annual average (because we are on a sphere, 
#   we need to weight the data by cos of latitude)

weight = np.cos(np.deg2rad(ncep_air.lat)) / np.cos(np.deg2rad(ncep_air.lat)).mean(dim='lat')
Tglobal = (ncep_air.air * weight).mean(dim=('lat','lon','time'))
print(Tglobal)

<xarray.DataArray (level: 17)>
array([ 15.179084  ,  11.207003  ,   7.8383274 ,   0.21994135,
        -6.4483433 , -14.888848  , -25.570469  , -39.36969   ,
       -46.797905  , -53.652245  , -60.56356   , -67.006065  ,
       -65.53293   , -61.48664   , -55.853584  , -51.593952  ,
       -43.21999   ], dtype=float32)
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0


In [6]:
#  Uncomment the line below to see the pressure levels of Tglobal.

#Tglobal.level

We're going to convert temperature to degrees Kelvin, using a handy list of pre-defined constants in climlab.constants

In [4]:
#  Handy set of constants available as part of climlab package.

climlab.constants.tempCtoK

273.15

In [12]:
#Add this constant to the temperature profile
Tglobal += climlab.constants.tempCtoK
print( Tglobal)

<xarray.DataArray (level: 17)>
array([288.329082, 284.357002, 280.988327, 273.369941, 266.701657, 258.261156,
       247.579533, 233.780315, 226.352092, 219.497765, 212.586449, 206.143952,
       207.617073, 211.663363, 217.296419, 221.556055, 229.930018])
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 600.0 500.0 400.0 ...


In [13]:
#  A handy re-usable routine for making a plot of the temperature profiles
#  We will plot temperatures with respect to log(pressure) to get a 
#   height-like coordinate

#  Function to define log-pressure
def zstar(lev):
    return -np.log(lev / climlab.constants.ps)

#  Plotting function
def plot_soundings(result_list, name_list, plot_obs=True, fixed_range=True):
    color_cycle=['r', 'g', 'b', 'y']
    fig, ax = plt.subplots(figsize=(9,9))
    if plot_obs:
        ax.plot(Tglobal, zstar(Tglobal.level), color='k', label='Observed')    
    for i, state in enumerate(result_list):
        Tatm = state['Tatm']
        lev = Tatm.domain.axes['lev'].points
        Ts = state['Ts']
        ax.plot(Tatm, zstar(lev), color=color_cycle[i], label=name_list[i])
        ax.plot(Ts, 0, 'o', markersize=12, color=color_cycle[i])
    #ax.invert_yaxis()
    yticks = np.array([1000., 750., 500., 250., 100., 50., 20., 10., 5.])
    ax.set_yticks(-np.log(yticks/1000.))
    ax.set_yticklabels(yticks)
    ax.set_xlabel('Temperature (K)', fontsize=14)
    ax.set_ylabel('Pressure (hPa)', fontsize=14)
    ax.grid()
    ax.legend()
    if fixed_range:
        ax.set_xlim([200, 300])
        ax.set_ylim(zstar(np.array([1000., 5.])))
    #ax2 = ax.twinx()
    
    return ax

In [16]:
#  The default will plot of observed temperature profile
plot_soundings([],[])

____________
<a id='section4'></a>

## II. A 30-layer model using the observed temperatures
____________



We want to create a 30-layer model that is initialized with the observed, globally averaged temperature profile. Let's get started.

In [41]:
#  Initialize a grey radiation model with 30 levels. 
#  Look back at Lab 2a and change the number of levels.

#col = 
#print(col)

In [46]:
#  Print out the levels (these levels are based on pressure)

#col.lev

Are these levels different from the levels for the observed temperature?

Indeed, they are. So, if we want to plot the observed temperature and 
the modelled temperature on the same y-axis, we have to interpolate the
observed temperature onto the model pressure levels.

In [56]:
#  Interpolate the observed temperatures to 30 evenly spaced pressure levels
lev = col.lev

#  We use np.interp to do a linear interpolation
Tinterp = np.interp(lev, np.flipud(Tglobal.level), np.flipud(Tglobal))
print(Tinterp)
#  Note: Need to 'flipud' because the interpolation routine 
#  needs the pressure data to be in increasing order

In [57]:
#  Initialize model with observed temperatures
col.Ts[:] = Tglobal[0]
col.Tatm[:] = Tinterp

In [59]:
#  Let's plot our initialized model tempatures
#  This should look just like the observations
result_list = [col.state]
name_list = ['Observed, Interpolated']
plot_soundings(result_list, name_list);

### Tune absorptivity to get observed OLR

Now, let's tune the absorptivity/emissivity to get the observed OLR, so that our interpolated profiles gives us a model result that looks like observations.

OLR depends on the absorptivity/emissivity, so we are going to loop over a range of possible $\epsilon$ 's to find one that matches the observed OLR.

In [60]:
#  Need to tune absorptivity/emissivity to get OLR = 238.5

#  Absorptivity/emissivity is somewhere between 0.01 and 0.1 for this model, 
#   so let's start with an array spanning this range.
epsarray = np.linspace(0.01, 0.1, 100)

#  and we will initialize the OLR to be zero
OLRarray = np.zeros_like(epsarray)

In [ ]:
#Let's loop over the absorptivity/emissivity and see what values of OLR 
#  we get for our 30-layer model.

for i in range(epsarray.size):
    col.subprocess['LW'].absorptivity = epsarray[i]
    col.compute_diagnostics()
    OLRarray[i] = col.OLR

In [61]:
#  Plot the OLR versus absorptivity/emissivity to get a visual of where the 
#   absorptivity falls when OLR is 238.5.

#plt.plot()
#plt.grid()
#plt.xlabel('epsilon')
#plt.ylabel('OLR')

We can be more precise with a numerical root-finder.

In [17]:
def OLRanom(eps):
    col.subprocess['LW'].absorptivity = eps
    col.compute_diagnostics()
    return col.OLR - 238.5

In [64]:
# Use numerical root-finding to get the equilibria
from scipy.optimize import brentq
#  brentq is a root-finding function
#  Need to give it a function and two end-points
#  It will look for a zero of the function between those end-points
eps = brentq(OLRanom, 0.01, 0.1)
print(eps)

Great! We get a value that corresponds to what our plot showed us.
Let's set the model absorptivity to this value.

In [66]:
#  Set new absorptivity.

#col.subprocess.LW.absorptivity = eps
#col.subprocess.LW.absorptivity

In [67]:
#  Check that this value gives you the observed OLR

#col.compute_diagnostics()
#col.OLR

____________
<a id='section6'></a>

## III. Radiative equilibrium in the 30-layer model
____________


In [73]:
#  Let's make a duplicate of our existing model to use to run out 
#   to radiative equilibrium.
re = climlab.process_like(col)

In [18]:
#  To get to equilibrium, we just time-step the model forward long enough. Let's integrate our model 1 year.

#re.integrate_years()

In [76]:
# Check for energy balance (ASR - OLR)


In [81]:
#  Plot the result
result_list.append(re.state)
name_list.append('Radiative equilibrium (grey gas)')
plot_soundings(result_list, name_list)

Some properties of the **radiative equilibrium** temperature profile:

- The surface is warmer than observed.
- The lower troposphere is colder than observed.
- Very cold air is sitting immediately above the warm surface.
- There is no tropopause, no stratosphere.

In [ ]:
#  Compare the RE model surface temperature (re.Ts) to the observed (Tglobal[0])


____________
<a id='section7'></a>

## IV. Radiative-Convective Equilibrium in the 30-layer model
____________

We recognize that the large drop in temperature just above the surface is unphysical. Parcels of air in direct contact with the ground will be warmed by mechansisms other than radiative transfer.

These warm air parcels will then become buoyant, and will convect upward, mixing their heat content with the environment.

We **parameterize** the statistical effects of this mixing through a **convective adjustment**. 

At each timestep, our model checks for any locations at which the **lapse rate** exceeds some threshold. Unstable layers are removed through an energy-conserving mixing formula.

This process is assumed to be fast relative to radiative heating. In the model, it is instantaneous.

### Add the convective adjustment as an additional subprocess

In [82]:
#  Here is the existing model
#print(re)

In [83]:
#  First we make a new clone
rce = climlab.process_like(re)

#  Then create a new ConvectiveAdjustment process. Note that we set the lapse_rate to be the dry adiabatic lapse rate.
conv = climlab.convection.ConvectiveAdjustment(state=rce.state,adj_lapse_rate=9.8)

#  And add it to our model
rce.add_subprocess('Convective Adjustment', conv)
print(rce)

This model is exactly like our previous models, except for one additional subprocess called ``Convective Adjustment``. 

We passed a parameter ``adj_lapse_rate`` (in K / km) that sets the neutrally stable lapse rate -- in this case, 9.8 K / km.

This number is chosed to very loosely represent the net effect of **dry convection**.

In [19]:
#  Run out to equilibrium. Integrate for 1 year.
rce.integrate_years()

In [20]:
#  Check for energy balance


In [80]:
#  Plot the result
result_list.append(rce.state)
name_list.append('Radiatve-Convective equilibrium (grey gas, dry lapse rate)')
plot_soundings(result_list, name_list)

Introducing convective adjustment into the model cools the surface quite a bit (compared to Radiative Equilibrium, in green here) -- and warms the lower troposphere. It gives us a MUCH better fit to observations.

____________
<a id='section3'></a>
## Questions
____________

1. a. How many pressure levels are there for the observed temperature profile? What is the pressure of the top level and the bottom level?
<p/>
   b. What is the pressure of the top level and bottom level of your 30-layer model? 
<p/>

2. a. How much warmer is your surface temperature after the model reaches radiative equilibrium compared to observed?
<p/>
   b. How does the surface temperature change after the model reaches radiative-convective equilibrium compared to observed?
<p/>

3. Re-run you radiative-convective model with the moist adiabatic lapse rate (6.5 K/km) rather than the dry adiabatic lapse rate. By how much does the surface temperature change compared to the simulation with the dry adiabatic lapse rate?
<p/>

4. Why do the RCE models produce a more realistic temperature profile than the RE model?
<p/>

5. What key upper atmospheric component is missing in our model that prevents our temperature profile from looking realistic? 
<p/>
</p>